In [ ]:
from langchain_core.documents import Document
from langchain.text_splitter import (RecursiveCharacterTextSplitter,
                                     CharacterTextSplitter,
                                     TokenTextSplitter)

In [2]:
##Example of document - page content + metadata
doc = Document(
    page_content='This is an example text file',
    metadata={
        "source":"example.txt",
        "page":1,
        "date_creation":"19/10/2025"
    }
)

print(doc.page_content)
print(doc.metadata)

This is an example text file
{'source': 'example.txt', 'page': 1, 'date_creation': '19/10/2025'}


In [8]:
from langchain_community.document_loaders import TextLoader
##Loading a text file here##
loader = TextLoader("data//text_files//machine_learning.txt",encoding='utf-8')
docs = loader.load()
print(f"Number of documents {len(docs)}")
print(f"Content preview:\n {docs[0].page_content[0:100]}")

Number of documents 1
Content preview:
 Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables system


In [ ]:
### Read all documents from a prticular directory###
##Disadvantage - Only one typr of files##
from langchain_community.document_loaders import DirectoryLoader
loader_dir = DirectoryLoader(path='data/text_files',
                             glob="**/*.txt", ##pattern for directory##
                             loader_cls = TextLoader,
                             loader_kwargs={'encoding':'utf-8'},
                             show_progress=True)
docs = loader_dir.load()
print("Number of documents loaded:",len(docs))8
for i,doc in enumerate(docs):
    print(f"\nDocument {i+1}:")
    print(f"Source: {doc.metadata['source']}")
    print(f"Length: {len(doc.page_content)}")

100%|██████████| 2/2 [00:00<00:00, 1055.57it/s]

Number of documents loaded: 2

Document 1:
Source: data\text_files\machine_learning.txt
Length: 575

Document 2:
Source: data\text_files\python_intro.txt
Length: 489
